In [ ]:
#import kaggle api
!pip install -q kaggle
!mkdir -p ~/.kaggle
!cp /content/kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
#importing the dataset
!kaggle datasets download -d sumanthvrao/daily-climate-time-series-data


Dataset URL: https://www.kaggle.com/datasets/sumanthvrao/daily-climate-time-series-data
License(s): CC0-1.0
daily-climate-time-series-data.zip: Skipping, found more recently modified local copy (use --force to force download)


In [ ]:
!unzip daily-climate-time-series-data


Archive:  daily-climate-time-series-data.zip
replace DailyDelhiClimateTest.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: DailyDelhiClimateTest.csv  
replace DailyDelhiClimateTrain.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: DailyDelhiClimateTrain.csv  


In [ ]:
!ls

daily-climate-time-series-data.zip  DailyDelhiClimateTrain.csv	sample_data
DailyDelhiClimateTest.csv	    kaggle.json


In [ ]:
#we do some preprocessing here
import pandas as pd
import builtins
from sklearn.preprocessing import MinMaxScaler
import numpy as np
df=pd.read_csv("DailyDelhiClimateTrain.csv")
print(df.head())
print("-----------------------------------------------------")
print(df.isnull().sum())
#wohoo no shitty data thank god
df['date']=pd.to_datetime(df['date'])
print("-----------------------------------------------------")
features=["meantemp","humidity","wind_speed","meanpressure"]
df=df[features].values
scaler=MinMaxScaler()
scaled_train_data=scaler.fit_transform(df)
print(scaled_train_data)
seq_len=30

#-----------------------PREPARING THE DATA TO BE TRAINED ON NOW----------------------------------------------

#the rnn is trained on (example btw)- shape(window of 3 elements,their output-i.e the next element)
#therefore it learns to predict the next element from the window of seq_len
#here the 31st element is the output element for the 1st iteration
def seq(scaled_train_data,seq_len):
  x_train=[]
  y_train=[]
  for i in builtins.range(len(scaled_train_data)-seq_len):
    x_train.append(scaled_train_data[i:i+seq_len])
    y_train.append(scaled_train_data[seq_len+i])
  return x_train,y_train
x_train,y_train=seq(scaled_train_data,seq_len)
x_train=np.array(x_train)
y_train=np.array(y_train)
print(x_train.shape)#input
print(y_train.shape)#output


         date   meantemp   humidity  wind_speed  meanpressure
0  2013-01-01  10.000000  84.500000    0.000000   1015.666667
1  2013-01-02   7.400000  92.000000    2.980000   1017.800000
2  2013-01-03   7.166667  87.000000    4.633333   1018.666667
3  2013-01-04   8.666667  71.333333    1.233333   1017.166667
4  2013-01-05   6.000000  86.833333    3.700000   1016.500000
-----------------------------------------------------
date            0
meantemp        0
humidity        0
wind_speed      0
meanpressure    0
dtype: int64
-----------------------------------------------------
[[0.12227074 0.8209571  0.         0.13260331]
 [0.04279476 0.90759076 0.07058266 0.132881  ]
 [0.0356623  0.84983498 0.10974262 0.13299381]
 ...
 [0.24745269 0.88063806 0.14842886 0.13289464]
 [0.276718   0.84983498 0.17349597 0.13265971]
 [0.12227074 1.         0.         0.1326467 ]]
(1432, 30, 4)
(1432, 4)


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
#defining the hyperparamter
num_layers=4
input_dim=4
output_dim=4
hidden_dim=128
batch_size=32
class TimeSeries(nn.Module):
    def __init__(self,num_layers,input_dim,output_dim,hidden_dim):
        super(TimeSeries,self).__init__()
        self.RNN=nn.RNN(input_dim,hidden_dim,num_layers,nonlinearity="relu",bias=True,batch_first=True,dropout=0.0,bidirectional=False,device=None,dtype=None)
        self.fc=nn.Linear(hidden_dim,1)
    def forward(self,x):
        #x_shape=(batch_size,seq_len,input_dim)
        out,_=self.RNN(x)
        final_out=out[:,-1,:]
        output=self.fc(final_out)
        return output



In [ ]:
#preparing test set
df_test=pd.read_csv("DailyDelhiClimateTest.csv")
print(df_test.head())
features=["meantemp","humidity","wind_speed","meanpressure"]
df_test=df_test[features].values
scaled_test_data=scaler.transform(df)
print(scaled_test_data)
x_test,y_test=seq(scaled_test_data,seq_len)
x_test=np.array(x_test)
y_test=np.array(y_test)
print(x_test.shape)
print(y_test.shape)


#splitting into test and validation sets
split_ratio=0.8
split_index=int(len(x_test)*split_ratio)
x_test,x_val=x_test[:split_index],x_test[split_index:]
y_test,y_val=y_test[:split_index],y_test[split_index:]

x_train=torch.from_numpy(x_train).float()
y_train=torch.from_numpy(y_train).float()
x_test=torch.from_numpy(x_test).float()
y_test=torch.from_numpy(y_test).float()
x_val=torch.from_numpy(x_val).float()
y_val=torch.from_numpy(y_val).float()

#making datasets and dataloaders
train_data=torch.utils.data.TensorDataset(x_train,y_train)
train_loader=torch.utils.data.DataLoader(train_data,batch_size=batch_size,shuffle=True)
val_data=torch.utils.data.TensorDataset(x_val,y_val)
val_loader=torch.utils.data.DataLoader(val_data,batch_size=batch_size,shuffle=True)
test_data=torch.utils.data.TensorDataset(x_test,y_test)
test_loader=torch.utils.data.DataLoader(test_data,batch_size=batch_size,shuffle=True)


         date   meantemp   humidity  wind_speed  meanpressure
0  2017-01-01  15.913043  85.869565    2.743478     59.000000
1  2017-01-02  18.500000  77.222222    2.894444   1018.277778
2  2017-01-03  17.111111  81.888889    4.016667   1018.333333
3  2017-01-04  18.700000  70.050000    4.545000   1015.700000
4  2017-01-05  18.388889  74.944444    3.300000   1014.333333
[[0.12227074 0.8209571  0.         0.13260331]
 [0.04279476 0.90759076 0.07058266 0.132881  ]
 [0.0356623  0.84983498 0.10974262 0.13299381]
 ...
 [0.24745269 0.88063806 0.14842886 0.13289464]
 [0.276718   0.84983498 0.17349597 0.13265971]
 [0.12227074 1.         0.         0.1326467 ]]
(1432, 30, 4)
(1432, 4)


In [ ]:
# Training loop
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
model = TimeSeries(num_layers, input_dim, output_dim, hidden_dim)
epochs = 100
lr = 0.001
lossfunc = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr)

print("Training started")

for epoch in range(epochs):
    model.train()
    train_loss = 0.0

    for inp, out in train_loader:
        optimizer.zero_grad()
        outputs = model(inp)
        loss_val = lossfunc(outputs, out)
        loss_val.backward()
        optimizer.step()
        train_loss += loss_val.item()

    train_loss /= len(train_loader)

    model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for inp, out in val_loader:
            outputs = model(inp)
            loss_val = lossfunc(outputs, out)
            val_loss += loss_val.item()

    val_loss /= len(val_loader)

    print(f"Epoch [{epoch+1}/{epochs}], Train Loss: {train_loss:.6f}, Val Loss: {val_loss:.6f}")

print("Training done woops")


cuda
Training started


/usr/local/lib/python3.11/dist-packages/torch/nn/modules/loss.py:610: UserWarning: Using a target size (torch.Size([16, 4])) that is different to the input size (torch.Size([16, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/usr/local/lib/python3.11/dist-packages/torch/nn/modules/loss.py:610: UserWarning: Using a target size (torch.Size([8, 4])) that is different to the input size (torch.Size([8, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/usr/local/lib/python3.11/dist-packages/torch/nn/modules/loss.py:610: UserWarning: Using a target size (torch.Size([15, 4])) that is different to the input size (torch.Size([15, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss

Epoch [1/100], Train Loss: 0.077133, Val Loss: 0.075120
Epoch [2/100], Train Loss: 0.068684, Val Loss: 0.074710
Epoch [3/100], Train Loss: 0.068463, Val Loss: 0.074748
Epoch [4/100], Train Loss: 0.068439, Val Loss: 0.074477
Epoch [5/100], Train Loss: 0.068539, Val Loss: 0.074580
Epoch [6/100], Train Loss: 0.068406, Val Loss: 0.074693
Epoch [7/100], Train Loss: 0.068411, Val Loss: 0.074697
Epoch [8/100], Train Loss: 0.068375, Val Loss: 0.074467
Epoch [9/100], Train Loss: 0.068366, Val Loss: 0.074461
Epoch [10/100], Train Loss: 0.068308, Val Loss: 0.074648
Epoch [11/100], Train Loss: 0.068477, Val Loss: 0.074632
Epoch [12/100], Train Loss: 0.068417, Val Loss: 0.074791
Epoch [13/100], Train Loss: 0.068417, Val Loss: 0.074425
Epoch [14/100], Train Loss: 0.068256, Val Loss: 0.074420
Epoch [15/100], Train Loss: 0.068245, Val Loss: 0.074442
Epoch [16/100], Train Loss: 0.068280, Val Loss: 0.074532
Epoch [17/100], Train Loss: 0.068303, Val Loss: 0.075025
Epoch [18/100], Train Loss: 0.068468, Va